In [3]:
import cv2
import numpy as np

# === Camera calibration parameters ===
# Replace these with your actual calibration results!
#camera_matrix = np.array([[800, 0, 320],
#                          [0, 800, 240],
#                          [0, 0, 1]], dtype=float)

#dist_coeffs = np.zeros((4, 1))  # Replace with real distortion values if available

data = np.load('camera_calibration.npz')
camera_matrix = data['camera_matrix']
dist_coeffs = data['dist_coeffs']

# === Marker settings ===
marker_length = 0.05  # meters (5 cm)

# === ArUco dictionary and detector setup ===
aruco_dict = cv2.aruco.getPredefinedDictionary(cv2.aruco.DICT_4X4_50)
parameters = cv2.aruco.DetectorParameters()

# === Video capture ===
cap = cv2.VideoCapture(0)

# === Distance history for smoothing ===
distance_history = []
MAX_HISTORY = 20
TILT_LIMIT_DEGREES = 45  # Max allowed marker tilt angle

def is_tilt_too_high(rvec):
    """Return True if marker is tilted beyond allowed limit."""
    rotation_vector = np.linalg.norm(rvec)
    angle_degrees = np.degrees(rotation_vector)
    return angle_degrees > TILT_LIMIT_DEGREES

while True:
    ret, frame = cap.read()
    if not ret:
        break

    corners, ids, _ = cv2.aruco.detectMarkers(frame, aruco_dict, parameters=parameters)

    if ids is not None and len(ids) >= 2:
        # Estimate pose of each marker
        rvecs, tvecs, _ = cv2.aruco.estimatePoseSingleMarkers(
            corners, marker_length, camera_matrix, dist_coeffs
        )

        valid_tvecs = []

        for i in range(len(ids)):
            cv2.aruco.drawDetectedMarkers(frame, corners)
            cv2.drawFrameAxes(frame, camera_matrix, dist_coeffs, rvecs[i], tvecs[i], 0.03)

            # Filter out tilted markers
            if not is_tilt_too_high(rvecs[i]):
                valid_tvecs.append((ids[i][0], tvecs[i][0]))

        # Sort by ID to ensure consistency
        valid_tvecs.sort()

        if len(valid_tvecs) >= 2:
            # Pick first two markers
            tvec1 = valid_tvecs[0][1]
            tvec2 = valid_tvecs[1][1]

            distance = np.linalg.norm(tvec1 - tvec2)

            # Add to history buffer
            distance_history.append(distance)
            if len(distance_history) > MAX_HISTORY:
                distance_history.pop(0)

            # Calculate smoothed average
            avg_distance = np.mean(distance_history)

            # Display both raw and averaged distance
            text = f"Avg Distance: {avg_distance*100:.2f} cm"
            cv2.putText(frame, text, (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 2)

    else:
        # No valid markers — clear history to avoid showing old values
        distance_history.clear()

    cv2.imshow("Stable Distance Measurement", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


KeyboardInterrupt: 